In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report  # Moved to top

# Paths (unchanged)
TRAIN_IMG_DIR = r'/media/prashant/12 TB HDD 21/cancer_portal/BreastCancer-YOLOv8.v1i (2).multiclass-20250626T082316Z-1-001/BreastCancer-YOLOv8.v1i.multiclass/train'
VAL_IMG_DIR = r'/media/prashant/12 TB HDD 21/cancer_portal/BreastCancer-YOLOv8.v1i (2).multiclass-20250626T082316Z-1-001/BreastCancer-YOLOv8.v1i.multiclass/valid'
TEST_IMG_DIR = r'/media/prashant/12 TB HDD 21/cancer_portal/BreastCancer-YOLOv8.v1i (2).multiclass-20250626T082316Z-1-001/BreastCancer-YOLOv8.v1i.multiclass/test'
TRAIN_CSV = r'/media/prashant/12 TB HDD 21/cancer_portal/BreastCancer-YOLOv8.v1i (2).multiclass-20250626T082316Z-1-001/BreastCancer-YOLOv8.v1i.multiclass/train/train.csv'
VAL_CSV = r'/media/prashant/12 TB HDD 21/cancer_portal/BreastCancer-YOLOv8.v1i (2).multiclass-20250626T082316Z-1-001/BreastCancer-YOLOv8.v1i.multiclass/valid/valid.csv'
TEST_CSV = r'/media/prashant/12 TB HDD 21/cancer_portal/BreastCancer-YOLOv8.v1i (2).multiclass-20250626T082316Z-1-001/BreastCancer-YOLOv8.v1i.multiclass/test/test.csv'

BATCH_SIZE = 32
NUM_EPOCHS = 200
TRAIN_AUGMENT_FACTOR = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {DEVICE}")
print(f"Batch size: {BATCH_SIZE}, Epochs: {NUM_EPOCHS}")
print(f"Augmentation factor: {TRAIN_AUGMENT_FACTOR}x\n")

class AugmentedProstateCancerDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None, augment=False):
        if not os.path.exists(csv_path):
            raise FileNotFoundError(f"CSV file not found: {csv_path}")
        self.df = pd.read_csv(csv_path)
        
        required_columns = ['filename', 'cancer']
        for col in required_columns:
            if col not in self.df.columns:
                raise ValueError(f"CSV missing required column: '{col}'")
        
        invalid_labels = self.df[~self.df['cancer'].isin([0, 1])]
        if not invalid_labels.empty:
            raise ValueError(f"Invalid labels found in CSV:\n{invalid_labels}")
        
        if not os.path.exists(img_dir):
            raise NotADirectoryError(f"Image directory not found: {img_dir}")
        
        self.img_dir = img_dir
        self.transform = transform
        self.augment = augment
        self.original_size = len(self.df)
        
        num_pos = (self.df['cancer'] == 1).sum()
        num_neg = (self.df['cancer'] == 0).sum()
        print(f"Dataset: {os.path.basename(csv_path)}")
        print(f"  Original samples: {self.original_size}")
        print(f"  Positive (cancer): {num_pos} ({num_pos/self.original_size*100:.1f}%)")
        print(f"  Negative (healthy): {num_neg} ({num_neg/self.original_size*100:.1f}%)")
        
        missing_files = []
        for idx, row in self.df.iterrows():
            img_path = os.path.join(self.img_dir, row['filename'])
            if not os.path.exists(img_path):
                missing_files.append((idx, row['filename']))
        
        if missing_files:
            print("\nWARNING: Missing image files:")
            for idx, fname in missing_files[:5]:
                print(f"  Row {idx}: {fname}")
            if len(missing_files) > 5:
                print(f"  ... and {len(missing_files)-5} more")
            raise FileNotFoundError(f"Total missing images: {len(missing_files)}")
        else:
            print("  All image files verified successfully")
        
    def __len__(self):
        if self.augment:
            return self.original_size * TRAIN_AUGMENT_FACTOR
        return self.original_size
    
    def __getitem__(self, idx):
        orig_idx = idx % self.original_size
        img_name = self.df.iloc[orig_idx]['filename']
        label = self.df.iloc[orig_idx]['cancer']
        img_path = os.path.join(self.img_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"\nERROR loading image: {img_path}")
            print(f"Exception: {str(e)}")
            image = Image.new('RGB', (224, 224), color='black')
            return image, torch.tensor(-1, dtype=torch.float32)
        
        # Fixed: Always use the provided transform
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.float32)

# Image Transformations (unchanged)
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create Datasets and DataLoaders (unchanged)
print("\n" + "="*50)
print("Creating training dataset with augmentation:")
train_dataset = AugmentedProstateCancerDataset(TRAIN_CSV, TRAIN_IMG_DIR, train_transform, augment=True)
print(f"  Augmented training size: {len(train_dataset)} images ({TRAIN_AUGMENT_FACTOR}x original)")

print("\n" + "="*50)
print("Creating validation dataset:")
val_dataset = AugmentedProstateCancerDataset(VAL_CSV, VAL_IMG_DIR, val_transform, augment=False)
print(f"  Validation size: {len(val_dataset)} images")

print("\n" + "="*50)
print("Creating test dataset:")
test_dataset = AugmentedProstateCancerDataset(TEST_CSV, TEST_IMG_DIR, val_transform, augment=False)
print(f"  Test size: {len(test_dataset)} images")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

print("\n" + "="*50)
print(f"Training batches: {len(train_loader)} ({len(train_dataset)} images)")
print(f"Validation batches: {len(val_loader)} ({len(val_dataset)} images)")
print(f"Test batches: {len(test_loader)} ({len(test_dataset)} images)")

# Initialize Model (unchanged)
print("\n" + "="*50)
print("Initializing ResNet50 model...")
model = models.resnet50(weights='DEFAULT')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)
model = model.to(DEVICE)

# Loss and Optimizer (unchanged)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3
)

print(f"\nModel architecture:")
print(model)
print(f"\nTrainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Function to visualize augmented images (unchanged)
def visualize_augmentations(dataset, num_samples=5):
    print("\nVisualizing augmented images...")
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, num_samples*3))
    fig.suptitle('Image Augmentation Examples', fontsize=16)
    
    for i in range(num_samples):
        orig_idx = np.random.randint(0, dataset.original_size)
        img_name = dataset.df.iloc[orig_idx]['filename']
        img_path = os.path.join(dataset.img_dir, img_name)
        orig_image = Image.open(img_path).convert('RGB')
        
        aug_idx1 = orig_idx * TRAIN_AUGMENT_FACTOR
        aug_idx2 = orig_idx * TRAIN_AUGMENT_FACTOR + 1
        
        aug_image1, _ = dataset[aug_idx1]
        aug_image2, _ = dataset[aug_idx2]
        
        def denormalize(tensor):
            mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
            std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
            tensor = tensor * std + mean
            tensor = tensor.clamp(0, 1)
            return tensor.permute(1, 2, 0).numpy()
        
        axes[i, 0].imshow(orig_image)
        axes[i, 0].set_title(f"Original\n{img_name}")
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(denormalize(aug_image1))
        axes[i, 1].set_title("Augmentation 1")
        axes[i, 1].axis('off')
        
        axes[i, 2].imshow(denormalize(aug_image2))
        axes[i, 2].set_title("Augmentation 2")
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig('augmentation_samples.png', dpi=150)
    plt.close()
    print("Saved augmentation_samples.png")

# Visualize augmentations (unchanged)
visualize_augmentations(train_dataset)

# Training Function with tqdm progress bars (unchanged)
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    start_time = time.time()
    
    print("\n" + "="*50)
    print("Starting training...")
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        # Training Phase
        model.train()
        running_loss = 0.0
        processed = 0
        
        # Wrap train loader with tqdm
        train_loop = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Training]', leave=False)
        for inputs, labels in train_loop:
            if (labels < 0).any():
                print("Skipping batch with invalid labels")
                continue
                
            inputs = inputs.to(DEVICE)
            labels = labels.unsqueeze(1).to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            processed += inputs.size(0)
            
            # Update progress bar
            train_loop.set_postfix(loss=loss.item())
            
        epoch_loss = running_loss / processed
        
        # Validation Phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        # Wrap validation loader with tqdm
        val_loop = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Validation]', leave=False)
        with torch.no_grad():
            for inputs, labels in val_loop:
                if (labels < 0).any():
                    print("Skipping batch with invalid labels")
                    continue
                    
                inputs = inputs.to(DEVICE)
                labels = labels.unsqueeze(1).to(DEVICE)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                
                preds = torch.sigmoid(outputs) > 0.5
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                # Update progress bar
                current_acc = correct / total if total > 0 else 0
                val_loop.set_postfix(acc=current_acc)
        
        val_loss = val_loss / total if total > 0 else 0
        val_acc = correct / total if total > 0 else 0
        
        # Update history
        history['train_loss'].append(epoch_loss)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Track LR changes
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_acc)
        new_lr = optimizer.param_groups[0]['lr']
        
        epoch_time = time.time() - epoch_start
        print(f"\nEpoch {epoch+1}/{num_epochs} ({epoch_time:.1f}s)")
        print(f"  Train Loss: {epoch_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        print(f"  Current LR: {old_lr:.6f}")
        
        # Print if LR changed
        if new_lr != old_lr:
            print(f"  LR reduced from {old_lr:.6f} to {new_lr:.6f}")
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, 'best_resnet50_augmented_model.pth')
            print(f"  Saved new best model with accuracy: {best_acc:.4f}")
    
    total_time = time.time() - start_time
    print(f"\nTraining complete in {total_time//60:.0f}m {total_time%60:.0f}s")
    print(f"Best Validation Accuracy: {best_acc:.4f}")
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

# Start training (unchanged)
trained_model, history = train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS)

# Final summary (unchanged)
print("\n" + "="*50)
print("Training Summary:")
print(f"Original training samples: {train_dataset.original_size}")
print(f"Augmented training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Final Validation Accuracy: {history['val_acc'][-1]:.4f}")
print(f"Best Validation Accuracy: {max(history['val_acc']):.4f}")
print(f"Model saved as: 'best_resnet50_augmented_model.pth'")

# Plot training history (unchanged)
def plot_history(history):
    plt.figure(figsize=(12, 5))
    
    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(history['val_acc'], label='Validation Accuracy', color='green')
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=150)
    plt.close()
    print("Saved training_history.png")

plot_history(history)

# Fixed test_model function (complete version)
def test_model(model, test_loader):
    print("\n" + "="*50)
    print("Testing best model on test dataset...")
    
    model.eval()
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    
    # Create progress bar for test phase
    test_loop = tqdm(test_loader, desc='Testing', leave=True)
    with torch.no_grad():
        for inputs, labels in test_loop:
            if (labels < 0).any():
                print("Skipping batch with invalid labels")
                continue
                
            inputs = inputs.to(DEVICE)
            labels = labels.unsqueeze(1).to(DEVICE)
            
            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5
            
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            # Store for confusion matrix
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds.extend(preds.cpu().numpy().flatten())
            
            # Update progress bar
            current_acc = correct / total if total > 0 else 0
            test_loop.set_postfix(acc=current_acc)
    
    test_acc = correct / total if total > 0 else 0
    print(f"\nTest Accuracy: {test_acc:.4f} ({correct}/{total})")
    
    # Generate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=['Healthy', 'Cancer'])
    
    print("\nConfusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(report)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['Healthy', 'Cancer'], rotation=45)
    plt.yticks(tick_marks, ['Healthy', 'Cancer'])
    
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('confusion_matrix.png', dpi=150)
    plt.close()
    print("Saved confusion_matrix.png")
    
    return test_acc

# Load best model and test (unchanged)
best_model = models.resnet50()
best_model.fc = nn.Linear(best_model.fc.in_features, 1)
best_model.load_state_dict(torch.load('best_resnet50_augmented_model.pth'))
best_model = best_model.to(DEVICE)

test_acc = test_model(best_model, test_loader)

# Final report (unchanged)
print("\n" + "="*50)
print("FINAL REPORT:")
print(f"Best Validation Accuracy: {max(history['val_acc']):.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("="*50)

Using device: cuda
Batch size: 32, Epochs: 200
Augmentation factor: 5x


Creating training dataset with augmentation:
Dataset: train.csv
  Original samples: 1281
  Positive (cancer): 1040 (81.2%)
  Negative (healthy): 241 (18.8%)
  All image files verified successfully
  Augmented training size: 6405 images (5x original)

Creating validation dataset:
Dataset: valid.csv
  Original samples: 240
  Positive (cancer): 171 (71.2%)
  Negative (healthy): 69 (28.7%)
  All image files verified successfully
  Validation size: 240 images

Creating test dataset:
Dataset: test.csv
  Original samples: 121
  Positive (cancer): 83 (68.6%)
  Negative (healthy): 38 (31.4%)
  All image files verified successfully
  Test size: 121 images

Training batches: 201 (6405 images)
Validation batches: 8 (240 images)
Test batches: 4 (121 images)

Initializing ResNet50 model...

Model architecture:
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64


Epoch 1/200 (35.6s)
  Train Loss: 0.1262
  Val Loss: 0.1461 | Val Acc: 0.9458
  Current LR: 0.001000
  Saved new best model with accuracy: 0.9458



Epoch 2/200 (35.3s)
  Train Loss: 0.0904
  Val Loss: 0.1634 | Val Acc: 0.9417
  Current LR: 0.001000



Epoch 3/200 (34.9s)
  Train Loss: 0.0937
  Val Loss: 0.1545 | Val Acc: 0.9417
  Current LR: 0.001000



Epoch 4/200 (34.4s)
  Train Loss: 0.0949
  Val Loss: 0.1540 | Val Acc: 0.9417
  Current LR: 0.001000


...


Epoch 199/200 (35.8s)
  Train Loss: 0.0337
  Val Loss: 0.2358 | Val Acc: 0.9250
  Current LR: 0.000000



Epoch 200/200 (35.5s)
  Train Loss: 0.0329
  Val Loss: 0.2348 | Val Acc: 0.9250
  Current LR: 0.000000

Training complete in 117m 18s
Best Validation Accuracy: 0.9542

Training Summary:
Original training samples: 1281
Augmented training samples: 6405
Validation samples: 240
Final Validation Accuracy: 0.9250
Best Validation Accuracy: 0.9542
Model saved as: 'best_resnet50_augmented_model.pth'
Saved training_history.png

Testing best model on test dataset...


Testing: 100%|██████████| 4/4 [00:00<00:00,  6.55it/s, acc=0.975]



Test Accuracy: 0.9752 (118/121)

Confusion Matrix:
[[38  0]
 [ 3 80]]

Classification Report:
              precision    recall  f1-score   support

     Healthy       0.93      1.00      0.96        38
      Cancer       1.00      0.96      0.98        83

    accuracy                           0.98       121
   macro avg       0.96      0.98      0.97       121
weighted avg       0.98      0.98      0.98       121

Saved confusion_matrix.png

FINAL REPORT:
Best Validation Accuracy: 0.9542
Test Accuracy: 0.9752


: 